In [2]:
import cvxpy as cp
import numpy as np
import mysql.connector
from itertools import combinations

# Fit a curve constraint

def curve_fit(coordinates1, coordinates2, coordinates3):
    y = np.array([coordinates1[1], coordinates2[1], coordinates3[1]])
    x = np.array([[coordinates1[0]**2, coordinates1[0], 1], 
                  [coordinates2[0]**2, coordinates2[0], 1],
                  [coordinates3[0]**2, coordinates3[0], 1]])
    constants = np.linalg.inv(x.T.dot(x)).dot(x.T).dot(y)
    return constants[0], constants[1], constants[2]

In [64]:
[a1, b1, c1] = curve_fit((84.85, 84.85), (0, 157.28), (-84.85, 84.85)) # shallow outfield
[a2, b2, c2] = curve_fit((-229.809, 229.809), (0, 400), (229.809, 229.809)) # outfield max

In [6]:
conn = mysql.connector.connect(user = 'r-user', password = 'h2p@4031',
                               host = 'saberbase.cn2snhhvsjfa.us-east-2.rds.amazonaws.com',
                               port = 3306,
                               database = 'figmentLeague')
cur = conn.cursor()
cur.execute("select ballpos_x, ballpos_y from rawFiltered where batterid = 605137") # Josh Bell
data = np.array(cur.fetchall())
conn.close()

In [105]:
positions = cp.Variable((7, 2))

objective = [cp.norm(cp.vec(data[:,0] - positions[i,0])) + cp.norm(cp.vec(data[:,1] - positions[i,1])) 
             for i in range(positions.shape[0])]

constraints = [positions[:,1] >= 63.64, # Everyone behind pitcher's mound
               positions[:,0] <= positions[:,1], # No one past right foul line
               -positions[:,0]<= positions[:,1], # No one past left foul line
               a1*cp.square(positions[0:4,0]) + b1*positions[0:4,0] + c1*np.ones((4,1))[0:5,0] >= positions[0:4,1],
               a2*cp.square(positions[:,0]) + b2*positions[:,0] + c2*np.ones((7,1))[:,0] >= positions[:,1],
               cp.square(positions[0,0] - 63.64) + cp.square(positions[0,1] - 63.74) <= 30**2, # 1B 30ft from 1st
              ]

prob = cp.Problem(cp.Minimize(cp.sum(objective)), constraints)

print('optimal value:', prob.solve())
print('Optimal 1B Position', positions[0,:].value)
print('Optimal 2B Position', positions[1,:].value)
print('Optimal 3B Position', positions[2,:].value)
print('Optimal Shortstop Position', positions[3,:].value)
print('Optimal Left Fielder Position', positions[4,:].value)
print('Optimal Center Fielder Position', positions[5,:].value)
print('Optimal Right Fielder Position', positions[6,:].value)

optimal value: 41197.88200577231
Optimal 1B Position [40.25726077 82.53487981]
Optimal 2B Position [ -5.40853287 138.62240476]
Optimal 3B Position [ -5.40853287 138.62240476]
Optimal Shortstop Position [ -5.40853287 138.62240476]
Optimal Left Fielder Position [ -5.40853104 138.6223936 ]
Optimal Center Fielder Position [ -5.40853104 138.6223936 ]
Optimal Right Fielder Position [ -5.40853104 138.6223936 ]


In [ ]:
# Constraints attempted with no luck:

cp.norm(cp.vec(positions[0,:] - positions[1,:])) >= 30**2 # Keeping players squared distances away from each other

a1*cp.square(positions[4:7,0]) + b1*positions[4:7,0] + c1*np.ones((3,1))[0:4,0] <= positions[4:7,1] # Keep OF in OF

# Objective function additions with no luck:

objective = objective + [cp.log(cp.abs(positions[i,0] - positions[j,0])) 
                         for i,j in combinations([0,1,2,3,4,5,6], 2)]



In [80]:
cp.norm(positions[0,:] - positions[1,:]) >= cp.square(30)

Inequality(Expression(CONSTANT, NONNEGATIVE, ()))

In [86]:
cp.log(cp.abs(positions[0,0] - positions[1,0]))

Expression(QUASICONVEX, UNKNOWN, ())

In [90]:
cp.norm(cp.vec(data[:,0] - positions[1,0]))

Expression(CONVEX, NONNEGATIVE, ())

In [96]:
attempt = cp.Variable(len(list(combinations([0,1,2,3,4,5,6], 2))), boolean=True)

In [100]:
attempt = [1 if cp.square(positions[i,0] - positions[j,0]) + cp.square(positions[i,1] - positions[j,1]) >= 30 else 0 for i,j in combinations([0,1,2,3,4,5,6], 2)]

Exception: Cannot evaluate the truth value of a constraint or chain constraints, e.g., 1 >= x >= 0.

In [104]:
attempt[0] = 1 if (cp.square(positions[0,0].value - positions[1,0].value) + \
             cp.square(positions[0,1].value - positions[1,1].value) >= 30**2) else 0

Exception: Cannot evaluate the truth value of a constraint or chain constraints, e.g., 1 >= x >= 0.